In [7]:
import pymongo
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [4]:
#setup splinter
executable_path={'executable_path': ChromeDriverManager().install()}
browser=Browser('chrome',**executable_path, headless=False)

  Using cached webdriver_manager-3.3.0-py2.py3-none-any.whl (16 kB)
  Using cached configparser-5.0.2-py3-none-any.whl (19 kB)
  Using cached crayons-0.4.0-py2.py3-none-any.whl (4.6 kB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install splinter

  Using cached splinter-0.14.0-py3-none-any.whl (36 kB)

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)
